# preprocessing

In [1]:
import MeCab, re, tqdm, json, glob
from collections import Counter
import os, random, re, csv, json, datetime
import mysql.connector
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot') 

In [2]:
# lemmatize
tagger = MeCab.Tagger()
def lemma(text:str) -> list:
    return [x.split(',')[6] for x in tagger.parse(text).split('\n')[:-2]]

def tokenize(text:str) -> list:
    """
    tokenize sentence into list of tokens with linguistic features
    0:surface form, 1:pos, 2:pos2, 3:pos3, 4:pos4, 5:conjugation type, 6:conjugation form, 7:lemma, 8:phono, 9:phone

    >>> tokenize('大きい家は走りたくなるな')
    [['大きい', '形容詞', '自立', '*', '*', '形容詞・イ段', '基本形', '大きい', 'オオキイ', 'オーキイ'],
    ['家', '名詞', '一般', '*', '*', '*', '*', '家', 'イエ', 'イエ'],
    ['は', '助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ'],
    ['走り', '動詞', '自立', '*', '*', '五段・ラ行', '連用形', '走る', 'ハシリ', 'ハシリ'],
    ['たく', '助動詞', '*', '*', '*', '特殊・タイ', '連用テ接続', 'たい', 'タク', 'タク'],
    ['なる', '動詞', '自立', '*', '*', '五段・ラ行', '基本形', 'なる', 'ナル', 'ナル'],
    ['な', '助詞', '終助詞', '*', '*', '*', '*', 'な', 'ナ', 'ナ']]
    """
    # exclude final two tokens : 'EOS', ''
    tokens = tagger.parse(text).split('\n')[:-2]
    tokens = [re.split(r'[,\t]', token) for token in tokens]
    surface, lemma, pos = [],[],[]
    for token in tokens:
        surface.append(token[0])
        lemma.append(token[7])
        pos.append(token[1])
    return surface, lemma, pos

In [83]:
l = tokenize('明日コロナにかかった')
l

(['明日', 'コロナ', 'に', 'かかっ', 'た'],
 ['明日', 'コロナ', 'に', 'かかる', 'た'],
 ['名詞', '名詞', '助詞', '動詞', '助動詞'])

In [ ]:
len(pd.read_json('nhkweb2018.json'))

In [92]:
# concat json files
data = pd.read_json('nhkweb2012.json')
for i in range(2013,2020):
    data = pd.concat([data, pd.read_json(f'nhkweb{i}.json')])

In [95]:
data = data[['id','title','genre','article','datePublished']]
data

,id,title,genre,article,datePublished
0,k10013902141000,“原発政策ころころ変わる”,[政治],野田総理大臣は、千葉県四街道市で街頭演説し、日本未来の党や日本維新の会が掲げる原子力政策につ...,2012-12-02T18:09
1,k10014002951000,六本木襲撃事件 １５人に逮捕状,[社会],ことし９月、東京・六本木のクラブで、客の男性が男らに金属バットのようなもので殴られて死亡した...,2012-12-06T16:13
2,k10014085401000,専門家“ミサイル内部の機器不具合か”,[社会],北朝鮮が発射を予告した事実上の弾道ミサイルを、１０日、発射しなかったことについて、海上自衛隊...,2012-12-10T15:51
3,k10014344461000,新華社“島巡る争いが試練”,[国際],中国国営の新華社通信は、ことしの中国外交について、日本政府による沖縄県の尖閣諸島の国有化や南...,2012-12-21T12:01
4,k10014441691000,鉄板直撃２人死亡 会社を捜索,[社会],２５日、広島県東広島市の国道を走っていた大型トレーラーの荷台から崩れ落ちた鉄板が乗用車を直撃...,2012-12-26T13:49
...,...,...,...,...,...
42352,k10012232931000,日本ハム 斎藤佑樹投手が結婚 相手は同い年の一般女性,[スポーツ],プロ野球・日本ハムは８年前のドラフト１位、31歳の斎藤佑樹投手が同い年の一般女性と結婚したこ...,2019-12-31T17:27:02+09:00
42353,k10012232941000,ＩＲ汚職 贈賄側は海外から1000万円超の現金 政界工作目的か,[社会],ＩＲ=統合型リゾート施設の事業をめぐって秋元司衆議院議員が逮捕された汚職事件で、贈賄側の中国...,2019-12-31T17:53:01+09:00
42354,k10012232951000,西日本豪雨で被災 仮設住宅に年越しそば届ける 広島 呉,[社会],去年７月の西日本豪雨で大きな被害を受けた広島県呉市では大みそかの31日、ボランティア団体が仮...,2019-12-31T19:05:20+09:00
42355,k10012232961000,ＷＢＯフライ級タイトルマッチ 田中が３回目の防衛,[スポーツ],ボクシング、ＷＢＯ＝世界ボクシング機構のフライ級タイトルマッチが東京で行われ、チャンピオンの...,2019-12-31T19:09:07+09:00


In [96]:
# make tokenized table
alllst = []
for line in data['article']:
    alllst.append(tokenize(line))

In [97]:
df = pd.DataFrame(alllst, columns=['tokens','lemma','pos'])
df

,tokens,lemma,pos
0,"[野田, 総理, 大臣, は, 、, 千葉, 県, 四街道, 市, で, 街頭, 演説, し...","[野田, 総理, 大臣, は, 、, 千葉, 県, 四街道, 市, で, 街頭, 演説, す...","[名詞, 名詞, 名詞, 助詞, 記号, 名詞, 名詞, 名詞, 名詞, 助詞, 名詞, 名..."
1,"[ことし, ９月, 、, 東京, ・, 六本木, の, クラブ, で, 、, 客, の, 男...","[ことし, ９月, 、, 東京, ・, 六本木, の, クラブ, で, 、, 客, の, 男...","[名詞, 名詞, 記号, 名詞, 記号, 名詞, 助詞, 名詞, 助詞, 記号, 名詞, 助..."
2,"[北朝鮮, が, 発射, を, 予告, し, た, 事実, 上, の, 弾道, ミサイル, ...","[北朝鮮, が, 発射, を, 予告, する, た, 事実, 上, の, 弾道, ミサイル,...","[名詞, 助詞, 名詞, 助詞, 名詞, 動詞, 助動詞, 名詞, 名詞, 助詞, 名詞, ..."
3,"[中国, 国営, の, 新華社通信, は, 、, ことし, の, 中国, 外交, について,...","[中国, 国営, の, 新華社通信, は, 、, ことし, の, 中国, 外交, について,...","[名詞, 名詞, 助詞, 名詞, 助詞, 記号, 名詞, 助詞, 名詞, 名詞, 助詞, 記..."
4,"[２, ５, 日, 、, 広島, 県, 東広島, 市, の, 国道, を, 走っ, て, い...","[２, ５, 日, 、, 広島, 県, 東広島, 市, の, 国道, を, 走る, て, い...","[名詞, 名詞, 名詞, 記号, 名詞, 名詞, 名詞, 名詞, 助詞, 名詞, 助詞, 動..."
...,...,...,...
97055,"[プロ, 野球, ・, 日本ハム, は, ８, 年, 前, の, ドラフト, １, 位, 、...","[プロ, 野球, ・, 日本ハム, は, ８, 年, 前, の, ドラフト, １, 位, 、...","[名詞, 名詞, 記号, 名詞, 助詞, 名詞, 名詞, 名詞, 助詞, 名詞, 名詞, 名..."
97056,"[ＩＲ, =, 統合, 型, リゾート, 施設, の, 事業, をめぐって, 秋元, 司, ...","[ＩＲ, *, 統合, 型, リゾート, 施設, の, 事業, をめぐって, 秋元, 司, ...","[名詞, 名詞, 名詞, 名詞, 名詞, 名詞, 助詞, 名詞, 助詞, 名詞, 名詞, 名..."
97057,"[去年, ７月, の, 西日本, 豪雨, で, 大きな, 被害, を, 受け, た, 広島,...","[去年, ７月, の, 西日本, 豪雨, で, 大きな, 被害, を, 受ける, た, 広島...","[名詞, 名詞, 助詞, 名詞, 名詞, 助詞, 連体詞, 名詞, 助詞, 動詞, 助動詞,..."
97058,"[ボクシング, 、, ＷＢＯ, ＝, 世界, ボクシング, 機構, の, フライ, 級, タ...","[ボクシング, 、, ＷＢＯ, ＝, 世界, ボクシング, 機構, の, フライ, 級, タ...","[名詞, 記号, 名詞, 記号, 名詞, 名詞, 名詞, 助詞, 名詞, 名詞, 名詞, 助..."


In [99]:
# concat tables
data = pd.concat([data.reset_index(), df], axis=1)

In [102]:
data.iloc[:,1:].to_csv('nhkweb.csv', index=None)

# load csv

In [3]:
DATA = pd.read_csv('nhkweb.csv')

In [12]:
DATA

,id,title,article
0,k10013902141000,“原発政策ころころ変わる”,野田総理大臣は、千葉県四街道市で街頭演説し、日本未来の党や日本維新の会が掲げる原子力政策につ...
1,k10014002951000,六本木襲撃事件 １５人に逮捕状,ことし９月、東京・六本木のクラブで、客の男性が男らに金属バットのようなもので殴られて死亡した...
2,k10014085401000,専門家“ミサイル内部の機器不具合か”,北朝鮮が発射を予告した事実上の弾道ミサイルを、１０日、発射しなかったことについて、海上自衛隊...
3,k10014344461000,新華社“島巡る争いが試練”,中国国営の新華社通信は、ことしの中国外交について、日本政府による沖縄県の尖閣諸島の国有化や南...
4,k10014441691000,鉄板直撃２人死亡 会社を捜索,２５日、広島県東広島市の国道を走っていた大型トレーラーの荷台から崩れ落ちた鉄板が乗用車を直撃...
...,...,...,...
97055,k10012232931000,日本ハム 斎藤佑樹投手が結婚 相手は同い年の一般女性,プロ野球・日本ハムは８年前のドラフト１位、31歳の斎藤佑樹投手が同い年の一般女性と結婚したこ...
97056,k10012232941000,ＩＲ汚職 贈賄側は海外から1000万円超の現金 政界工作目的か,ＩＲ=統合型リゾート施設の事業をめぐって秋元司衆議院議員が逮捕された汚職事件で、贈賄側の中国...
97057,k10012232951000,西日本豪雨で被災 仮設住宅に年越しそば届ける 広島 呉,去年７月の西日本豪雨で大きな被害を受けた広島県呉市では大みそかの31日、ボランティア団体が仮...
97058,k10012232961000,ＷＢＯフライ級タイトルマッチ 田中が３回目の防衛,ボクシング、ＷＢＯ＝世界ボクシング機構のフライ級タイトルマッチが東京で行われ、チャンピオンの...


In [6]:
genre = Counter()
for genres in DATA.genre:
    for g in eval(genres):
        genre[g] += 1
genre.most_common()

[('社会', 26837),
 ('国際', 25302),
 ('スポーツ', 14700),
 ('政治', 13136),
 ('ビジネス', 12473),
 ('気象・災害', 9897),
 ('科学・文化', 6470),
 ('暮らし', 3677),
 ('地域', 3093)]

In [15]:
pd.DataFrame(genre.most_common(), columns=['genre','count'])

,genre,count
0,社会,26837
1,国際,25302
2,スポーツ,14700
3,政治,13136
4,ビジネス,12473
5,気象・災害,9897
6,科学・文化,6470
7,暮らし,3677
8,地域,3093


In [12]:
# kango + する
KANJI = re.compile(r'[\u3400-\u4DBF\u4E00-\u9FFF\uF900-\uFAFF]{2,10}')
KATAKANA = re.compile(r'[ァ-ヶ]{1,2}')

def suru(genre=None, filename=None):
    count = Counter()
    df = {}
    if genre != None and type(genre) == str:
        genre = [genre]
    genre = set(genre)
    
    for i, row in DATA.iterrows():
        if not genre <= set(eval(row.genre)):
            continue
        lemmas = eval(row.lemma)
        for j, lemma in enumerate(lemmas):
            if j > 1 and lemma == 'する' and re.match(KANJI,lemmas[j-1]) and re.match(KANJI,lemmas[j-2]):
                word = ''.join(lemmas[j-2:j+1])
                count[word] += 1
                df[word] = df.get(word, []) + [i]
            elif j > 0 and lemma == 'する' and re.match(KANJI,lemmas[j-1]):
                word = ''.join(lemmas[j-1:j+1])
                count[word] += 1
                df[word] = df.get(word, []) + [i]
            
    return count, df

def katakana(genre=None):
    count = Counter()
    df = {}
    if genre != None and type(genre) == str:
        genre = [genre]
    genre = set(genre)
    
    for i, row in DATA.iterrows():
        if not genre <= set(eval(row.genre)):
            continue
        lemmas = eval(row.lemma)
        for j, lemma in enumerate(lemmas):
            if re.match(KATAKANA,lemma):
                count[lemma] += 1
                df[lemma] = df.get(lemma, []) + [i]
                
    return count, df

def get_df(total, word):
    return round(len(set(df[word])) / total * 100, 2)

# count

In [79]:
pd.DataFrame(genre.most_common(), columns=['genre','count'])

,genre,count
0,社会,26837
1,国際,25302
2,スポーツ,14700
3,政治,13136
4,ビジネス,12473
5,気象・災害,9897
6,科学・文化,6470
7,暮らし,3677
8,地域,3093


In [25]:
count, df = suru(genre='地域')
total = 3093

print(sum(count.values()))

for w,c in count.most_common(200):
    print(f'{w},{c},{get_df(total, w)}')

18777
参加する,886,17.26
確認する,409,8.76
設置する,330,6.66
展示する,266,5.01
販売する,258,5.24
披露する,254,5.82
公開する,244,4.66
発表する,212,4.24
再開する,191,3.3
利用する,189,4.49
検討する,175,4.14
紹介する,163,3.85
撮影する,155,3.36
水揚げする,153,2.52
完成する,152,3.59
用意する,151,4.3
指定する,139,3.36
期待する,134,3.62
出席する,131,3.3
運行する,124,2.62
開発する,116,2.23
被災する,111,2.36
活用する,108,2.65
提供する,107,2.39
整備する,107,2.23
登録する,106,2.07
生息する,105,2.42
出発する,104,2.88
解除する,104,1.71
飼育する,103,2.33
説明する,100,2.88
発信する,100,2.46
導入する,99,1.88
出荷する,99,2.3
協力する,97,2.52
企画する,97,2.62
成長する,94,2.23
代表する,94,2.65
登場する,93,2.49
開催する,92,2.0
到着する,92,2.13
応援する,90,2.17
支援する,86,2.04
死亡する,84,1.78
栽培する,84,2.04
収穫する,83,1.94
制作する,82,1.84
感謝する,79,2.07
祈願する,77,1.84
再現する,77,1.71
感動する,75,2.1
注目する,74,1.81
管理する,73,1.97
避難する,71,1.58
記念する,71,1.78
体験する,69,1.55
表現する,69,1.88
獲得する,69,1.33
出場する,67,1.65
優勝する,66,1.36
発生する,65,1.71
挑戦する,64,1.78
担当する,63,1.65
逮捕する,63,0.91
主催する,62,1.84
予定する,62,1.84
誕生する,60,1.42
捕獲する,60,1.03
開花する,58,1.23
開通する,57,0.87
指摘する,56,1.33
観測する,56,1.13
保護する,55,1.16
運営する,54,1.49
保管する

36067535

In [44]:
DATA.loc[:1000][['id','title','article','datePublished']].to_json('nhksmall.json', force_ascii=False, orient='records', lines=True)

In [47]:
config = {'user': 'nozomi',
        'password': 'database',
        'host': 'chula.cd7wwaejjsg3.ap-southeast-1.rds.amazonaws.com',
        'database': 'jtdic'}


# connect & insert
cn = mysql.connector.connect(**config)
cursor = cn.cursor()

In [52]:
word = '男子中学生'
cursor.execute(f"SELECT DISTINCT title, article FROM nhksmall WHERE article LIKE '%{word}%'")
result = list(cursor) # [[title, article],,]
if result == []:
    reply = 'หาไม่เจอในคลังข้อมูลครับ\n(พิมพ์ help จะแสดงวิธีใช้)'
else:
    if len(result) > 5:
        result = random.sample(result, 5)
    reply = ''
    pattern = re.compile(f'(?:^|[、。])([^、。]*?{word}[^、。]*?(?:[、。]|$))')
    for title, article in result:
        candidates = re.findall(pattern, article)
        reply += '・' + random.choice(candidates) + '\n\n'
    reply = reply.strip()

In [53]:
re.findall(pattern, article)

['川崎市の河川敷で男子中学生が殺害され、']

# export for SQL

In [7]:
DATA = DATA[['id','title','article']]
DATA['article'] = DATA['article'].apply(lambda x:re.sub(r'\n+','',x))

In [9]:
DATA.to_csv('nhkweb.csv',index=None)

In [11]:
!wc -l nhkweb.csv

   97061 nhkweb.csv
